[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/028_pytorch_nn/pytorch_nn.ipynb)

# Pytorch - Redes Neuronales

En el post [anterior](https://sensioai.com/blog/027_pytorch_intro) hicimos una introducción al framework de `redes neuronales` `Pytorch`. Hablamos de sus tres elementos fundamentales: el objeto `tensor` (similar al `array` de `NumPy`) `autograd` (que nos permite calcular derivadas de manera automáticas) y el soporte GPU. En este post vamos a entrar en detalle en la  funcionalidad que nos ofrece la librería para diseñar redes neuronales de manera flexible.

In [2]:
import torch
import pandas as pd
from sklearn.preprocessing import LabelEncoder

## Modelos secuenciales

La forma más sencilla de definir una `red neuronal` en `Pytorch` es utilizando la clase `Sequentail`. Esta clase nos permite definir una secuencia de capas, que se aplicarán de manera secuencial (las salidas de una capa serán la entrada de la siguiente). Ésto ya lo conocemos de posts anteriores, ya que es la forma ideal de definir un `Perceptrón Multicapa`.

In [3]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# D_in, H1, H2, D_out = 784, 100, 50, 10
# model = torch.nn.Sequential(
#     torch.nn.Linear(D_in, H1),
#     torch.nn.ReLU(),
#     torch.nn.Linear(H1, H2),
#     torch.nn.ReLU(),
#     torch.nn.Linear(H2, D_out),
# )


El modelo anterior es un `MLP` con 784 entradas, 100 neuronas en la capa oculta y 10 salidas. Podemos usar este modelo para hacer un clasificador de imágenes con el dataset MNIST. Pero primero, vamos a ver como podemos calcular las salidas del modelo a partir de unas entradas de ejemplo.

In [4]:
outputs = model(torch.randn(64, 784))
outputs.shape

torch.Size([64, 10])

In [5]:
print(outputs[0][:])

tensor([-0.3348, -0.0037,  0.2422,  0.3595,  0.0650,  0.0322, -0.0223,  0.0968,
         0.2215, -0.0772], grad_fn=<SliceBackward0>)


Como puedes ver, simplemente le pasamos los inputs al modelo (llamándolo como una función). En este caso, usamos un tensor con 64 vectores de 784 valores. Es importante remarcar que los modelos de `Pytorch` (por lo general) siempre esperan que la primera dimensión sea la dimensión *batch*. Si queremos entrenar esta red en una GPU, es tan sencillo como

In [6]:
model.to("cuda")

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=10, bias=True)
)

Vamos a ver ahora como entrenar este modelo con el dataset MNIST.

In [7]:
# Reemplaza 'dataset.csv' con la ruta correcta a tu archivo CSV
data = pd.read_csv('Glyphs_updated.csv')

# Eliminar todas las columnas con valores NaN
data = data.dropna(axis=1)

# Llenar los valores faltantes con un valor específico, como cero (0)
data = data.fillna(0)

Y = data['label']
X = data.drop(columns=['label'])

X = X.apply(pd.to_numeric, errors='coerce')
X = X.fillna(0)

X.shape, Y.shape

((54894, 784), (54894,))

In [8]:
import numpy as np

# normalización y split
import numpy as np
x_2=np.array(X)
y_2= Y


# normalización y split

X_train =x_2[:36982] / 255.
X_test =x_2[17912:] / 255.
y_train = y_2[:36982]
y_test = y_2[:17912]



# X_train, X_test, y_train, y_test = X[:60000] / 255., X[60000:] / 255., Y[:60000].astype(np.float32), Y[60000:].astype(np.float32)

In [9]:
# función de pérdida y derivada

def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

def cross_entropy(output, target):
    logits = output[torch.arange(len(output)), target]
    loss = - logits + torch.log(torch.sum(torch.exp(output), axis=-1))
    loss = loss.mean()
    return loss

In [10]:
torch.cuda.is_available()

True

In [11]:
print(X)

       1  2  3  4  5  6  7  8  9  10  ...  775  776  777  778  779  780  781  \
0      0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
1      0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
2      0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
3      0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
4      0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
...   .. .. .. .. .. .. .. .. ..  ..  ...  ...  ...  ...  ...  ...  ...  ...   
54889  0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
54890  0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
54891  0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
54892  0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
54893  0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   

       782  783  784  
0        0    0 

In [12]:
# convertimos datos a tensores y copiamos en gpu

label_encoder = LabelEncoder()
y_train_numeric = label_encoder.fit_transform(y_train)

X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.LongTensor(y_train_numeric)

# bucle entrenamiento
epochs = 100
lr = 0.8
log_each = 10
l = []
for e in range(1, epochs + 1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = cross_entropy(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    model.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad
    torch.cuda.empty_cache()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

RuntimeError: ignored

Como puedes observar en el ejemplo, podemos calcular la salida del modelo con una simple línea. Luego calculamos la función de pérdida, y llamando a la función `backward` `Pytorch` se encarga de calcular las derivadas de la misma con respecto a todos los parámetros del modelo automáticamente (si no queremos acumular estos gradientes, nos aseguramos de llamar a la función `zero_grad` para ponerlos a cero antes de calcularlos). Por útlimo, podemos iterar por los parámetros del modelo aplicando la regla de actualización deseada (en este caso usamos `descenso por gradiente`).

In [ ]:
from sklearn.metrics import accuracy_score

def evaluate(x):
    model.eval()
    y_pred = model(x)
    y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

0.9603015075376884

De esta manera, tenemos mucha flexibilidad para definir nuestras redes.

## Resumen